In [137]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [138]:
df = pd.read_csv("../data/carprice_clean.csv")
df.head(15)

,carburant,turbo,type_vehicule,roues_motrices,emplacement_moteur,longueur_voiture(cm),largeur_voiture(cm),poids_vehicule(kg),type_moteur,nombre_cylindres,taille_moteur,systeme_carburant,chevaux,consommation_ville(L/100km),consommation_autoroute(L/100km),prix,marque,modele
0,gas,std,décapotable,propulsion,avant,428.8,162.8,1155.8,dohc,four,130,mpfi,111,11.200714,8.711667,13495.000,alfa-romeo,giulia
1,gas,std,décapotable,propulsion,avant,428.8,162.8,1155.8,dohc,four,130,mpfi,111,11.200714,8.711667,16500.000,alfa-romeo,stelvio
2,gas,std,hayon,propulsion,avant,434.8,166.4,1280.5,ohcv,six,152,mpfi,154,12.379737,9.046731,16500.000,alfa-romeo,Quadrifoglio
3,gas,std,berline,traction,avant,448.6,168.1,1060.0,ohc,four,109,mpfi,102,9.800625,7.840500,13950.000,audi,100ls
4,gas,std,berline,quatre_roues_motrices,avant,448.6,168.7,1280.9,ohc,five,136,mpfi,115,13.067500,10.691591,17450.000,audi,100ls
5,gas,std,berline,traction,avant,450.3,168.4,1137.2,ohc,five,136,mpfi,110,12.379737,9.408600,15250.000,audi,fox
6,gas,std,berline,traction,avant,489.5,181.4,1290.0,ohc,five,136,mpfi,110,12.379737,9.408600,17710.000,audi,100ls
7,gas,std,break,traction,avant,489.5,181.4,1339.9,ohc,five,136,mpfi,110,12.379737,9.408600,18920.000,audi,5000
8,gas,turbo,berline,traction,avant,489.5,181.4,1399.8,ohc,five,131,mpfi,140,13.836176,11.760750,23875.000,audi,4000
9,gas,turbo,hayon,quatre_roues_motrices,avant,452.6,172.5,1384.8,ohc,five,131,mpfi,160,14.700938,10.691591,17859.167,audi,5000s(diesel)


In [139]:
from sklearn.model_selection import train_test_split

# Initialisation de l'estimateur

y = df["prix"]
X = df.drop("prix", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

from sklearn.preprocessing import MinMaxScaler # pour axe x et axe y , avoir une valeur de 0 a 1     x - min / min - max

# Sélectionner les colonnes numériques
numeric_cols = ['longueur_voiture(cm)', 'largeur_voiture(cm)', 'poids_vehicule(kg)', 'chevaux', 'consommation_ville(L/100km)', 'consommation_autoroute(L/100km)']

# Créer une instance de MinMaxScaler
scaler = MinMaxScaler()

# Normaliser les données d'entraînement
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

# Normaliser les données de test
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])


from sklearn.preprocessing import OneHotEncoder

# Sélectionner les colonnes catégorielles
categorial_cols = ["carburant", "turbo", "type_vehicule", "roues_motrices", "emplacement_moteur", "type_moteur", "nombre_cylindres", "systeme_carburant", "marque", "modele"]

# Créer une instance de OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=True, handle_unknown="ignore")

# Appliquer le One-Hot Encoding aux colonnes catégorielles
X_train_cat = onehot_encoder.fit_transform(X_train[categorial_cols])
X_test_cat = onehot_encoder.transform(X_test[categorial_cols])


# a l'aide de la classe ColumnTransformer, 
# on déclare à quelles variables on applique quel transformer
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_cols),
        ('cat', onehot_encoder, categorial_cols)
    ]
)

/home/apprenant/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Model LinearRegression

In [140]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# Initialiser le modèle de régression linéaire
linear_regression = LinearRegression()

# Combiner le préprocesseur avec le modèle de régression linéaire
model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', linear_regression)])

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

# Évaluer les performances du modèle sur les données de test
score = model.score(X_test, y_test)

print("R2 score:", score)

R2 score: 0.7795387560471124


/home/apprenant/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Model RandomForestRegressor

In [141]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Initialiser le modèle de RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Combiner le préprocesseur avec le modèle de RandomForestRegressor
model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', rf)])

# Définir la grille des hyperparamètres
param_grid = {
    'regressor__n_estimators': [200],
    'regressor__max_depth': [10],
    'regressor__min_samples_split': [5],
    'regressor__min_samples_leaf': [2]
}

# Rechercher les meilleurs hyperparamètres avec GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Évaluer les performances du modèle sur les données de test
score = grid_search.score(X_test, y_test)
print("R2 score:", score)

# Afficher les meilleurs hyperparamètres trouvés
print("Best hyperparameters:", grid_search.best_params_)


# Prédire les valeurs de la variable cible pour les données de test
y_pred = grid_search.predict(X_test)

# Calculer les métriques de régression
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Afficher les métriques
print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("R2 score:", r2)



/home/apprenant/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/apprenant/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/apprenant/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/apprenant/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868

R2 score: 0.842138020201997
Best hyperparameters: {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 200}
Mean squared error: 10770011.699391171
Mean absolute error: 2029.3612561947853
R2 score: 0.842138020201997


## Model Gradient Boosting

Il s'agit d'un algorithme de boosting qui combine plusieurs modèles de régression pour améliorer la prédiction. Il est très efficace pour les données tabulaires et peut souvent surpasser les autres algorithmes de régression.

In [157]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle


# Initialiser le modèle Gradient Boosting
gbr = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=3, random_state=42)

# Combiner le préprocesseur avec le modèle Gradient Boosting
model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', gbr)])

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

# Prédire les valeurs de la variable cible pour les données de test
y_pred = model.predict(X_test)

# Calculer les métriques de régression
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False) # obtenir la racine carrée du MSE
r2 = r2_score(y_test, y_pred)

# Afficher les métriques
print("Mean squared error:", mse)
print("Root Mean squared error:", rmse)
print("Mean absolute error:", mae)
print("R2 score:", r2)

# Enregistrer le modèle sur le disque
filename = '../models/model_carprice.pkl'
pickle.dump(model, open(filename, 'wb'))


/home/apprenant/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Mean squared error: 8289949.076387632
Root Mean squared error: 2879.2271665132002
Mean absolute error: 1824.390183584607
R2 score: 0.8784896609075032


In [149]:
import pandas as pd

# Prédire les valeurs de la variable cible pour les données de test
y_pred = model.predict(X_test)

# Créer un dataframe avec les prix réels et les prix prédits
df_prix_predict = pd.DataFrame({'prix_reel': y_test, 'prix_pred': y_pred})

# Afficher les 10 premières lignes du dataframe
df_prix_predict.head(20)


,prix_reel,prix_pred
15,30760.000,27907.898686
9,17859.167,23618.325893
115,16630.000,14606.093295
78,6669.000,6432.946176
66,18344.000,12790.019666
45,8916.500,5800.987615
143,11259.000,13506.004633
177,15998.000,13905.363926
200,21485.000,15423.272891
180,7775.000,10073.132003
